#### Importing libraries 

In [1]:
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from urllib.parse import urljoin, urlparse
import itertools
import zipfile
import xlrd
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

#### Web scrapping & Cleaning

In [ ]:
lista=[20100101,20110101,20120101,20130101,20140101,20150101,20160101,20170101,20180101,20190101]
lista2=[]

for i in lista:
    url = f'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/29005?date={i}'
    respuesta = requests.get(url)
    datos = respuesta.json()
    data=pd.DataFrame.from_records(datos)
    
    data['Fecha']=data['Data'].apply(lambda x: [next(iter(d.values())) for d in x])
    data['Año']=data['Data'].apply(lambda x:[d.get('Anyo') for d in x])
    data['Población']=data['Data'].apply(lambda x:[d.get('Valor') for d in x])
    
    data['Fecha']=data['Fecha'].apply(lambda x: re.sub('\]','',str(x)))
    data['Fecha']=data['Fecha'].apply(lambda x: re.sub('\[','',str(x)))
    
    data['Fecha']=data['Fecha'].apply(lambda x: str(x).strip())
    data['Fecha']=pd.to_numeric(data['Fecha'], errors='ignore')
    data['Fecha'].fillna(0, inplace=True)
    data['Fecha'].apply(int)
    
    data['Fecha']=data['Fecha'].apply(lambda x: datetime.date.fromtimestamp(
                      int(x) // 1000))
    
    data['Año']=data['Año'].apply(lambda x: re.sub('\]','',str(x)))
    data['Año']=data['Año'].apply(lambda x: re.sub('\[','',str(x)))
    
    data['Población']=data['Población'].apply(lambda x: re.sub('\]','',str(x)))
    data['Población']=data['Población'].apply(lambda x: re.sub('\[','',str(x)))
        
    data.drop(columns=['Data','FK_Escala','FK_Unidad'])
    
    lista2.append(data)
    
    total=pd.concat(lista2)
    filtering=total['Año']==''
    
    total=total[~filtering]
    total=total.drop(columns=['Data','FK_Escala','FK_Unidad','COD'])

    total['Sexo']= total.Nombre.str.contains(pat = '(Mujeres.)', regex = True) 
    total['Sexo']=total['Sexo'].replace(True,'Mujeres')
    
    total['Sexo']=np.where(total['Nombre'].str.contains('Hombres'), "Hombres", total['Sexo'])
    total['Sexo']=total['Sexo'].replace(False,'Total')
    
    municipios=[]
    for i in total.Nombre.str.split('.'):
        municipios.append(i[0])
    
    total['Nombre']=municipios

/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [ ]:
total.shape

In [ ]:
total

In [2]:
url = f'https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfCodPostal/2018/jrubikf241580c2986609e03ee3216d79d3f457701c254e.html'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')

IRPF=[]
table=list(soup.find_all('tr'))
for y in table:
    city=re.findall('(</a>.*</th>)|(depth_div_2".*</div>)',str(y))
    citytop=re.findall('(</a>.*</th>)',str(y))

    if len(citytop)>0:
        fill='Resto'+' '+str([i for i in citytop])
    for item in itertools.chain.from_iterable(city):
        if ">Resto</div>" in str(item):
            IRPF.append([fill,re.findall('(table01_c3 table01_f.*?</td)',str(y))])
        else:
            IRPF.append([city,re.findall('(table01_c3 table01_f.*?</td)',str(y))])
            
columns=['City','Av.Disposable_Income']
data = [i for i in IRPF]
df = pd.DataFrame(data[2:], columns=columns)
df['City']=df['City'].apply(lambda x: re.sub('\</a>','',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('á','a',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('é','e',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('í','i',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('ó','o',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('ú','u',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('ñ','n',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('\</th>','',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('depth_div_2">|</div>','',str(x)))
df['City']=df['City'].apply(lambda x: re.sub(r'- ',' ',str(x)))
df['City']=df['City'].apply(lambda x: re.sub(r'-',' ',str(x)))
df['City']=df['City'].apply(lambda x: re.sub(r'\.','  ',str(x)))
df['City']=df['City'].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '',str(x)))
df['City']=df['City'].apply(lambda x: re.sub(r'  ',r'.',str(x)))

df['ZipCode']=df['City'].apply(lambda x: re.findall('\d+',str(x)))
df['ZipCode']=df['ZipCode'].apply(lambda x: re.sub('[^0-9]','',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('[\d+]{3,20}','',str(x)))
df['City']=df['City'].apply(lambda x: re.sub('[\d+]{3,20}','',str(x)))
df['City']=df['City'].apply(lambda x: x.strip())

df['Av.Disposable_Income']=df['Av.Disposable_Income'].apply(lambda x: re.sub(r'table01_c3.*>','',str(x)))
df['Av.Disposable_Income']=df['Av.Disposable_Income'].apply(lambda x: re.sub('\D+','',str(x)))

dfR=df[df['City'].str.contains('Resto')]
dfR=dfR[dfR['City']!='Resto']
dfR['City']=dfR['City'].apply(lambda x: re.sub('Resto ','',str(x)))

df=df[~df['City'].str.contains('Resto')]

df=df.drop_duplicates()

df['merge']=pd.to_numeric(df['ZipCode'])

In [3]:
dfR['City']=dfR['City'].apply(lambda x: re.sub('Coruna A','A Coruña',str(x)))
dfR['City']=dfR['City'].apply(lambda x: re.sub('AlicanteAlacant','Alicante',str(x)))
dfR['City']=dfR['City'].apply(lambda x: re.sub('ElcheElx','Elche',str(x)))
dfR['City']=dfR['City'].apply(lambda x: re.sub('Castellon de la PlanaCastello de la Plana','Castellon de la Plana',str(x)))
dfR['City']=dfR['City'].apply(lambda x: str(x).upper())
dfR['City']=dfR['City'].apply(lambda x: re.sub('LOGRONO','LOGROÑO',str(x)))
dfR['City']=dfR['City'].apply(lambda x: re.sub('LOGRONO','LOGROÑO',str(x)))
dfR['City']=dfR['City'].apply(lambda x: re.sub('SANT CUGAT DEL VALLS','S. CUGAT DEL VALLES',str(x)))
dfR.reset_index(drop=True,inplace=True)

In [4]:
data_market=pd.read_csv(f'..//Final_Project/Altadis/data_market.csv')
data_market=data_market.merge(df,left_on='ZipCode',right_on='merge', how='left')
data_market.drop(columns=['ZipCode_x','merge','City_y'])

,City_x,Domestic/Border,Name,TOTAL DUCADOS,TOTAL DUCADOS RUBIO,TOTAL FORTUNA,TOTAL ITG FMC,TOTAL NEGROS ITG,TOTAL NOBEL,TOTAL RUBIOS ITG,...,MS ITG FMC,MS ITG RYO,MS ITG PIPE,MS ITG CIGARS,FYMS ITG FMC,FYMS ITG RYO,FYMS ITG PIPE,FYMS ITG CIGARS,Av.Disposable_Income,ZipCode_y
0,TORREMOLINOS,TR-2 B,TORREMOLINOS-002,310800,566250,833224,7048874,377000,386600,6671874,...,0.375770,0.580277,0.181694,0.104185,0.336823,0.368322,0.037626,0.347646,NaN,NaN
1,ZAMORA -12,DOM,ZAMORA-012,430250,345292,129712,1272254,472850,220000,799404,...,0.290601,0.321186,0.000000,0.430870,0.336823,0.368322,0.037626,0.347646,NaN,NaN
2,ONDARROA,DOM,ONDARROA-003,520800,245160,3800,868960,612200,0,256760,...,0.205546,0.098846,NaN,0.243079,0.336823,0.368322,0.037626,0.347646,NaN,NaN
3,VALMASEDA,DOM,BALMASEDA-003,366800,69600,17000,550800,420000,11200,130800,...,0.225239,0.265120,NaN,0.332792,0.336823,0.368322,0.037626,0.347646,NaN,NaN
4,VALMASEDA,DOM,BALMASEDA-001,55600,10200,3400,86800,67000,1200,19800,...,0.264796,0.188053,NaN,0.561538,0.336823,0.368322,0.037626,0.347646,NaN,NaN
5,VALMASEDA,DOM,BALMASEDA-002,654400,135800,6000,906800,742600,3000,164200,...,0.152056,0.269286,0.023923,0.250281,0.336823,0.368322,0.037626,0.347646,NaN,NaN
6,CARRANZA,DOM,AMBASAGUAS,671000,76000,200,769600,693400,0,76200,...,0.293092,0.044554,NaN,0.275296,0.336823,0.368322,0.037626,0.347646,NaN,NaN
7,CARRANZA,DOM,CONCHE,354000,70600,3000,447600,372600,1000,75000,...,0.250028,0.031127,NaN,0.306578,0.336823,0.368322,0.037626,0.347646,NaN,NaN
8,EL MOLINAR,DOM,EL MOLINAR,72600,6200,0,84800,78600,0,6200,...,0.287263,NaN,NaN,0.077882,0.336823,0.368322,0.037626,0.347646,NaN,NaN
9,VENTORRO GALDAMES,DOM,VENTORRO-GALDAMES,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0.000000,0.336823,0.368322,0.037626,0.347646,NaN,NaN


In [5]:
lista=[]
for i in enumerate(dfR['City']):
    lista.append(i)

for x in lista:
    for i in data_market['City_x'].unique():
        if i==x[1]:
            value=(dfR[dfR['City']==x[1]]['Av.Disposable_Income'].loc[x[0]])
            data_market['Av.Disposable_Income']=np.where(((pd.isnull(data_market['Av.Disposable_Income']) & (data_market['City_x']==x[1]))), value, data_market['Av.Disposable_Income'])

for x in lista:
    for i in data_market['province'].unique():
        if i==x[1]:
            value=(dfR[dfR['City']==x[1]]['Av.Disposable_Income'].loc[x[0]])
            data_market['Av.Disposable_Income']=np.where(((pd.isnull(data_market['Av.Disposable_Income']) & (data_market['province']==x[1]))), value, data_market['Av.Disposable_Income'])

In [13]:
(data_market[data_market['Av.Disposable_Income'].isnull()]['province']).unique()

array(['ZAMORA', 'VIZCAYA', 'SEGOVIA', 'LUGO', 'TOLEDO', 'SORIA',
       'TARRAGONA', 'TERUEL', 'CIUDAD REAL', 'PONTEVEDRA', 'SALAMANCA',
       'CANTABRIA', 'ASTURIAS', 'PALENCIA', 'ORENSE', 'NAVARRA', 'LERIDA',
       'LA RIOJA', 'CADIZ', 'LEON', 'JAEN', 'HUESCA', 'HUELVA',
       'GUIPUZCOA', 'GUADALAJARA', 'CUENCA', 'GERONA', 'LA CORUÑA',
       'CASTELLON', 'CACERES', 'BALEARES', 'AVILA', 'BADAJOZ', 'ALAVA',
       'LA CORUÐA'], dtype=object)

In [41]:
si=[data_market['province']=='LUGO']
si1=(data_market['Av.Disposable_Income']).isnull()

In [ ]:
data_market[si&si1]

In [39]:
(data_market[data_market['province']=='LUGO']['customer_id']

0

VIZCAYA
SEGOVIA
LUGO